Proteomic and genomic analysis by tumor (including code for Supp. Fig. 3B)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
import pyreadr
from operator import itemgetter
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from statsmodels.stats import multitest

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            


tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

tumor_protein_dfs=[]
for tumor,rois in tumor_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df_tumor=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor})
    tumor_protein_dfs.append(cur_df_tumor)
protein_dat_by_tumor=pd.concat(tumor_protein_dfs,1)

protein_dat_by_tumor=protein_dat_by_tumor.T

In [ ]:
filenames = glob.glob("FISH_data/table*")
dfs_o = {}
for filename in filenames:
    df_o = pd.read_csv(filename)
    shreya_core_num = int(filename.split('_')[2])
    image_num = int(filename.split('_')[3].split('.')[0])
    dfs_o[(shreya_core_num,image_num)]=df_o

    
HI=6
MAX_HI = 50

def geno_hi(geno,df):
    return (df[geno]>=HI)&(df[geno]<=MAX_HI)
def geno_lo(geno,df):
    return (df[geno]>=1)&(df[geno]<HI)

for df_fish in list(dfs_o.values()):
    df_fish['EGFR_amp']=1*(geno_hi('EGFR',df_fish))
    df_fish['PDGFRA_amp']=1*(geno_hi('PDGFRA',df_fish))
    df_fish['CDK4_amp']=1*(geno_hi('CDK4',df_fish))
    
dfs={}    
for key,df in dfs_o.items():
        
    if key[0] not in tumor_core_to_shreya.values():
        continue

    
    df['EGFR_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']>=HI))
    df['EGFR_NOT_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']<HI))
    df['CDK4_NOT_EGFR_amp']=1*((df['CDK4']>=HI)&(df['EGFR']<HI))
    
    df['Non_amp']=1*((df['EGFR']<HI)&(df['CDK4']<HI))
    
    dfs[key]=df[(df['EGFR']<=MAX_HI)&(df['PDGFRA']<=MAX_HI)&(df['CDK4']<=MAX_HI)]

        
shreya_to_tumor={v:k[0] for k,v in tumor_core_to_shreya.items()}
core_list=list(set([item[0] for item in dfs.keys()]))
tumor_list = list(set([shreya_to_tumor[core] for core in core_list]))

tumor_image_dict={}
for k in dfs.keys():
    this_tumor=shreya_to_tumor[k[0]]
    if this_tumor in tumor_image_dict.keys():
        tumor_image_dict[this_tumor]=tumor_image_dict[this_tumor]+[k]
    else:
        tumor_image_dict[this_tumor]=[k]
        
dfs_in_tumor={}
for tumor, images in tumor_image_dict.items():
    dfs_in_tumor[tumor]=pd.concat([dfs[image] for image in images])

In [ ]:
tumor_props_dict={}
for tumor in tumor_list:
    this_df=dfs_in_tumor[tumor]
    E_only=len(this_df[this_df['EGFR_NOT_CDK4_amp']==1])/len(this_df)
    C_only=len(this_df[this_df['CDK4_NOT_EGFR_amp']==1])/len(this_df)
    dual=len(this_df[this_df['EGFR_CDK4_amp']==1])/len(this_df)
    Non_amp=len(this_df[this_df['Non_amp']==1])/len(this_df)
    tumor_props_dict[tumor]=[E_only,C_only,dual,Non_amp]

In [ ]:
amp_prop_df=pd.DataFrame.from_dict(tumor_props_dict,orient='index').rename(columns={0:'E_only',1:'C_only',2:'E_C',3:'None'})

In [ ]:
pd.concat([amp_prop_df,protein_dat_by_tumor],1).corr()

In [ ]:
def arr_to_1d_lis(arr):
    return list(itertools.chain(*list(arr)))
def list_to_arr(lst,length):
    return [lst[x:x+length] for x in range(0, len(lst), length)]
def p_value_adjustment(p_results):
    pvals1D=arr_to_1d_lis(p_results.to_numpy())
    adj_p=multitest.fdrcorrection(pvals1D)[1]
    p_results_adj=pd.DataFrame(list_to_arr(adj_p,len(p_results.columns)))
    p_results_adj=p_results_adj.set_index(p_results.index)
    p_results_adj=p_results_adj.rename(columns={list(p_results_adj.columns)[i]:list(p_results.columns)[i] for i in range(len(p_results.columns))})
    return p_results_adj
def get_stats():
    #geno_cols=['EGFR_only_amp','CDK4_only_amp','Dual_amp']#,'No_amp']
    df_dict={col:np.nan for col in ['None']}
    protein_list=list(protein_dat_by_tumor.columns)
    df_dict['Proteins']=protein_list
    coeff_results=pd.DataFrame.from_dict(df_dict).set_index('Proteins')
    pval_results=pd.DataFrame.from_dict(df_dict).set_index('Proteins')    
    for prot in protein_list:
        [coeff,pval]=list(stats.spearmanr(list(amp_prop_df['None']),list(protein_dat_by_tumor[prot])))
        coeff_results.loc[prot,'None']=coeff
        pval_results.loc[prot,'None']=pval
    adj_p=p_value_adjustment(pval_results)
    return coeff_results,pval_results,adj_p
    #return coeff_results,pval_results

In [ ]:
spearman_NA,pNA,adjp_NA=get_stats()

In [ ]:
pNA.sort_values(by='None')

In [ ]:
adjp_NA.sort_values(by='None')

In [ ]:
df_results=pd.concat([spearman_NA.rename(columns={'None':'Spearman correlation'}),adjp_NA.rename(columns={'None':'Adjusted p-value'})],1)

In [ ]:
df_results['-log(adj. p-value)']=-np.log(df_results['Adjusted p-value'])

In [ ]:
df_results=df_results.sort_values(by='Spearman correlation',ascending=False)

In [ ]:
top_bot_prots=pd.concat([df_results.head(5),df_results.tail(5)])
top_bot_prots['Protein']=top_bot_prots.index

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = 'Arial'
fig = plt.figure(figsize=(14,6))
g=sns.relplot(x="Protein", y="Spearman correlation", data=top_bot_prots, size='-log(adj. p-value)',color='black',sizes=(50, 500), aspect=1.5)
plt.xticks(rotation=90)
plt.grid(color='lightgray')
leg = g._legend
for t in leg.texts:
    # truncate label text to 4 characters
    t.set_text(t.get_text()[:4])
#plt.tight_layout()
#plt.savefig('prot_correlation_with_NA_tumor_wise.pdf')
plt.show()

In [ ]:
spearman_NA.loc['Olig2']

In [ ]:
spearman_NA.loc['EGFR']

In [ ]:
spearman_NA.loc['Sox2']

In [ ]:
NA_prot_correlations={}
for prot in protein_dat_by_tumor_rescaled.columns:
    NA_prot_correlations[prot]=stats.spearmanr(list(protein_dat_by_tumor[prot]),list(amp_prop_df['None']))

In [ ]:
NA_prot_cor_df=pd.DataFrame.from_dict(NA_prot_correlations,orient='index').sort_values(by='correlation',ascending=False)
NA_prot_cor_df['Protein']=NA_prot_cor_df.index
NA_prot_cor_df=NA_prot_cor_df.rename(columns={'correlation':'Spearman correlation'})
top_bot_prots=pd.concat([NA_prot_cor_df.head(5),NA_prot_cor_df.tail(10)])
top_bot_prots['-log(p-value)']=-np.log(top_bot_prots['pvalue'])